In [1]:
from imports import *
from functions import *
from constants import *
# Active l'extension autoreload dans un notebook Jupyter
%load_ext autoreload
%autoreload 2

# Launching Driver and Website Main Page

In [2]:
driver = create_driver()
driver.get(website_url)
wait = WebDriverWait(driver, timeout=3)
mi_cuenta = wait.until(EC.presence_of_element_located((By.XPATH, "//span[normalize-space()='Mi cuenta']")))  # Remplacer par l'élément attendu
mi_cuenta.click()
# Envoie de l'email sur le site
try:
    ingresar_email_zone = wait.until(EC.presence_of_element_located((By.XPATH, "//input[@placeholder='Ingresa tu email']")))
    ingresar_email_zone.clear()
    ingresar_email_zone.send_keys(EMAIL)
    ingresar_email_zone.send_keys(Keys.ENTER)
except:
    print('NO')

Waiting Function statement

In [3]:
wait10 = WebDriverWait(driver, timeout=10)

# Search Mechanic

In [114]:
SEARCH_QUERY = "portatil ASUS Ryzen 7 1TB 16GB"

Dynamically Building the Name of the CSV File to store collected Data

In [115]:
csv_output_name = f"Exito_Search_{SEARCH_QUERY.replace(" ", "_")}.csv"
csv_output_name

'Exito_Search_portatil_ASUS_Ryzen_7_1TB_16GB.csv'

In [21]:
main_page=driver.current_url
try:
    search_field = wait10.until(
        EC.presence_of_element_located((By.XPATH, "//input[contains(@placeholder, 'Buscar en exito.com') and contains(@data-fs-input, 'true')]"))
    )
    print("Search field found !")
    try:
        search_field.send_keys(SEARCH_QUERY)
        time.sleep(1)
        search_field.send_keys(Keys.ENTER)

    except Exception as e:
        print(str(e))

except:
    print("Search field not found")

Search field found !


In [ ]:
def filter_products_img_urls(img_urls):
    """
    Filtre une liste d'URLs d'images pour ne garder que celles qui contiennent **tous** les termes définis dans 'list_accepted_terms'.
    """
    list_accepted_terms = ["vtexassets", "arquivos", "ids", "v="]
    
    # Filtrer et garder uniquement les URLs qui contiennent **tous** les termes nécessaires
    filtered_urls = [url for url in img_urls if all(term in url for term in list_accepted_terms)]
    
    return filtered_urls

def normalize_text(text):
    # Supprimer les sauts de ligne et les espaces inutiles
    text = text.replace("\n", " ")  # Remplacer les sauts de ligne par un espace
    text = re.sub(r'\s+', ' ', text)  # Remplacer les espaces multiples par un seul espace
    text = text.strip()  # Supprimer les espaces en début et fin de texte
    return text

# Data Collection

In [158]:
from datetime import datetime as dt

In [ ]:
import logging
from datetime import datetime as dt
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.exceptions import NoSuchElementException

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

dict_results = {}
today = dt.now()
dict_results["Timestamp"] = today
count = 0
page_num = 1

while True:
    # Récupérer la page de produits
    time_sleep(page=page_num, action="navigate", page_type="listing")
    logging.info("🚀 Navigating to product listing page...")
    
    try:
        product_listing = wait10.until(EC.presence_of_element_located((By.XPATH, "//ul[contains(@data-fs-product-grid, 'true')]")))
        cards = product_listing.find_elements(By.XPATH, ".//div[contains(@class, 'productCard_contentInfo')]")
        total_results_str = wait10.until(EC.presence_of_element_located((By.XPATH, "//p[contains(@data-testid, 'total-product-count')]"))).text
        match = re.search(r"\d{1,9}", total_results_str)
        
        if match:
            total_results = int(match.group())
            dict_results["Total Result"] = total_results
            logging.info(f"🔢 Total products found: {total_results}")

    except Exception as e:
        logging.error(f"❌ Error while fetching the product listing: {e}")
        break

    # Boucle sur chaque produit
    for card in cards:
        count += 1
        datas = {}

        ####################### GET TITLE ##########################################################################################################################################################################################
        
        try:
            time_sleep(page=page_num, action="scroll", page_type="listing")
            logging.info("📝 Scraping product title...")
            title = wait10.until(EC.presence_of_element_located((By.XPATH, ".//h3[contains(@class, 'styles_name__qQJiK')]")))
            datas["Title"] = title.text
            logging.info(f"📦 Product Title: {title.text}")

            actual_price = card.find_element(By.XPATH, ".//p[contains(@class, 'ProductPrice')]").text
            datas["Price"] = actual_price
            logging.info(f"💰 Current Price: {actual_price}")

        ######################## GET PRICE AND DISCOUNTS ##########################################################################################################################################################################################
        
            try:
                last_price = card.find_element(By.XPATH, ".//p[contains(@class, 'promotion_price')]").text
                datas["Last Price"] = last_price
                logging.info(f"💸 Last Price: {last_price}")
                cleaned_actual_price = actual_price.replace('$', '').replace(',', '').strip()
                cleaned_last_price = last_price.replace('$', '').replace(',', '').strip()
                discount_value = get_discount_percentage(cleaned_last_price, cleaned_actual_price)
                datas["Discount"] = discount_value
                logging.info(f"🔻 Discount: {discount_value}%")
            except NoSuchElementException:
                logging.info("❌ No promotion available for this product.")

            try:
                images_zone = card.find_element(By.XPATH, ".//img[contains(@alt, 'Imagen del producto')]")
                img_url = images_zone.get_attribute('src')
                datas["Original Image"] = img_url
                logging.info(f"🖼️ Original Image URL: {img_url}")
                ActionChains(driver).move_to_element(images_zone).perform()
                new_img_url = images_zone.get_attribute('src')
                if new_img_url != img_url:
                    datas["Hovered Image"] = new_img_url
                    logging.info(f"🔄 Hovered Image URL: {new_img_url}")
                else:
                    logging.info("🖼️ No change in image after hover.")
            except NoSuchElementException:
                logging.info("❌ Image not found.")

            ########## PRODUCT PAGE SCRAPING ##########

            logging.info("🔗 Navigating to product detail page...")
            link_producto_detalles = find_element(card, "xpath",".//a[contains(@data-testid,'product-link')]").get_attribute('href')
            time_sleep(page=page_num, action="click", page_type="listing")
            driver.get(link_producto_detalles)
            time_sleep(page=page_num, action="navigate", page_type="product")
            product_section = wait10.until(EC.presence_of_element_located((By.XPATH, "//section[contains(@class, 'section product-details')]")))

            PLU_ID = find_element(product_section, "xpath", ".//span[contains(@class, 'product-title__specification')]").text
            datas["PLU_ID"] = PLU_ID
            logging.info(f"🆔 PLU ID: {PLU_ID}")

            imgs_tags = find_elements(product_section, "xpath", ".//img[contains(@src, '')]")
            imgs_url = [img.get_attribute('src') for img in imgs_tags]
            imgs_url = filter_products_img_urls(imgs_url)
            datas['Images URL'] = imgs_url
            logging.info(f"🖼️ Product Images: {imgs_url}")

            stock_details = find_element(product_section, "xpath", ".//p[@data-fs-product-details-stock__qty='true']").text
            datas["Stock Details"] = stock_details
            logging.info(f"📦 Stock Status: {stock_details}")

            dealer_part = find_element(product_section, "xpath", ".//a[contains(text(), 'Mundo digital vm')]")
            dealer = dealer_part.text
            datas["Dealer"] = dealer
            dealer_url = dealer_part.get_attribute("href")
            datas["Dealer URL"] = dealer_url
            logging.info(f"🏢 Dealer: {dealer}, URL: {dealer_url}")

            article_text_container = find_element(product_section, "xpath", ".//article[contains(@data-accordion-item, 'true') and contains(@data-testid, 'store-accordion-item-item')]")
            product_specifications = find_element(article_text_container, "xpath", ".//div[contains(@data-fs-content-specification,'true')]")
            detalles = find_elements(product_specifications, "xpath", ".//div[contains(@data-fs-specification,'')]")
            for div in detalles:
                paragraph_title = find_element(div, "xpath", ".//p[contains(@data-fs-title-specification, 'true')]").text
                paragraph_text = find_element(div, "xpath", ".//p[contains(@data-fs-text-specification, 'true')]").text
                detail = f"{paragraph_title}: {paragraph_text}"
                logging.info(f"📋 Product Specification: {detail}")

            product_description = find_element(article_text_container, "xpath", ".//div[contains(@data-fs-description-text, '')]").text
            datas["Product Description"] = product_description
            logging.info(f"📝 Product Description: {product_description}")

            delivery_details_zone = find_element(product_section, "xpath", ".//section[contains(@class, 'deliveryPromises')]")
            delivery_details = find_element(delivery_details_zone, "xpath", ".//span[contains(@data-fs-promises-bold,'true')]").text
            disponibilidad_compra_y_recoge = find_element(delivery_details_zone, "xpath", ".//p[contains(@data-fs-promises-bold,'true')]").text
            datas["Delivery Details"] = delivery_details
            datas["Purchase & Pick-up"] = disponibilidad_compra_y_recoge
            logging.info(f"🚚 Delivery Info: {delivery_details}")
            logging.info(f"🏪 Availability for Pick-up: {disponibilidad_compra_y_recoge}")

            dict_results[count] = datas

            driver.back()
            time_sleep(page=page_num, action="navigate", page_type="listing")

        except Exception as e:
            logging.error(f"❌ Error occurred while scraping product: {e}")
            break

    try:
        time_sleep(page=page_num, action="click", page_type="listing")
        button_next_page = wait10.until(EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'Pagination_next')]")))
        button_next_page.click()
        page_num += 1
        logging.info(f"📄 Moving to page {page_num}...")
    except NoSuchElementException:
        logging.info("🚫 No next page found. Stopping the loop...")
        break


5 articles found on the first page

🛍️ Portatil Asus Intel Core I5 1235U Ram 16Gb Ssd 1Tb Pantalla 15.6 Fhd Quiet Blue Color
💰 Prix actuel : $ 2.498.000
💸 Ancien prix : $ 3.449.000
🔻 Réduction : 28%%
🖼️ Image originale: https://exitocol.vtexassets.com/arquivos/ids/23178710-300-auto/portatil-asus-intel-core-i5-1235u-ram-16gb-ssd-1tb-pantalla-156-fhd-quiet-blue-color.jpg?v=638539797549700000
🔄 Image après survol: https://exitocol.vtexassets.com/arquivos/ids/23178711-300-auto/portatil-asus-intel-core-i5-1235u-ram-16gb-ssd-1tb-pantalla-156-fhd-quiet-blue-color.jpg?v=638539797551400000

🛍️ Portatil Asus Amd Ryzen 7 7730U Ssd 1Tb Ram 16Gb Led 16 Full Hd
💰 Prix actuel : $ 2.879.900
💸 Ancien prix : $ 3.009.800
🔻 Réduction : 4%%
🖼️ Image originale: https://exitocol.vtexassets.com/arquivos/ids/23717525-300-auto/portatil-asus-amd-ryzen-7-7730u-ssd-1tb-ram-16gb-led-16-full-hd.jpg?v=638569370397670000
🔄 Image après survol: https://exitocol.vtexassets.com/arquivos/ids/23717526-300-auto/portatil-asus

In [ ]:
total_results_str = wait10.until(EC.presence_of_element_located((By.XPATH, "//p[contains(@data-testid, 'total-product-count')]"))).text
match = re.search(r"\d{1,9}", total_results_str)

if match:
    total_results = int(match.group())
    print(f"Total Result : {total_results}")

493 resultados
Total Result : 493


In [ ]:
driver.get("https://www.exito.com/s?q=portatil+ASUS+Ryzen+7+1TB+RAM&sort=score_desc&page=0")

product_listing = wait10.until(EC.presence_of_element_located((By.XPATH, "//ul[contains(@data-fs-product-grid, 'true')]")))
cards = product_listing.find_elements(By.XPATH, ".//div[contains(@class, 'productCard_contentInfo')]")

one_card = cards[0]

# Article Details

In [ ]:
##########  PRODUCT PAGE SCRAPING ###########################################################################################################################################################################################################################

product_section = find_element(driver, "xpath", "//section[contains(@class, 'section product-details')]")

PLU_ID = find_element(product_section, "xpath", ".//span[contains(@class, 'product-title__specification')]").text

# Récupération des images
imgs_tags = find_elements(product_section, "xpath", ".//img[contains(@src, '')]")

imgs_url = [img.get_attribute('src') for img in imgs_tags]

imgs_url =filter_products_img_urls(imgs_url)
print(imgs_url)

stock_details = find_element(product_section, "xpath", ".//p[@data-fs-product-details-stock__qty='true']").text

# Affiche le texte
print(f"Texte complet : {stock_details}")


# Trouver l'élément <a> contenant "Mundo digital vm"
dealer_part = find_element(product_section, "xpath", ".//a[contains(text(), 'Mundo digital vm')]")

# Récupérer le texte de l'élément <a> (le texte "Mundo digital vm")
dealer = dealer_part.text
print(f"Dealer: {dealer}")

# Récupérer l'URL associée à cet élément <a>
dealer_url = dealer_part.get_attribute("href")
print(f"Dealer URL : {dealer_url}")


article_text_container = find_element(product_section, 
    "xpath", ".//article[contains(@data-accordion-item, 'true') and contains(@data-testid, 'store-accordion-item-item')]")

product_specifications = find_element(article_text_container, 
            "xpath", ".//div[contains(@data-fs-content-specification,'true')]")
detalles = find_elements(product_specifications, 
        "xpath", ".//div[contains(@data-fs-specification,'')]")
len(detalles)
for div in detalles:
    paragraph_title = find_element(div, "xpath", ".//p[contains(@data-fs-title-specification, 'true')]").text
    paragraph_text = find_element(div, "xpath", ".//p[contains(@data-fs-text-specification, 'true')]").text
    detail = f"{paragraph_title}: {paragraph_text}"
    print(detail)
product_description = find_element(article_text_container, "xpath", 
                    ".//div[contains(@data-fs-description-text, '')]").text

normalized_description = normalize_text(product_description)
print(normalized_description)

delivery_details_zone = find_element(product_section, "xpath", 
".//section[contains(@class, 'deliveryPromises')]")
delivery_details = find_element(delivery_details_zone, "xpath",
            ".//span[contains(@data-fs-promises-bold,'true')]").text
disponibilidad_compra_y_recoge = find_element(delivery_details_zone, "xpath",
            ".//p[contains(@data-fs-promises-bold,'true')]").text

##########  ###########################################################################################################################################################################################################################



['https://exitocol.vtexassets.com/arquivos/ids/23178710/portatil-asus-intel-core-i5-1235u-ram-16gb-ssd-1tb-pantalla-156-fhd-quiet-blue-color.jpg?v=638539797549700000', 'https://exitocol.vtexassets.com/arquivos/ids/23178711/portatil-asus-intel-core-i5-1235u-ram-16gb-ssd-1tb-pantalla-156-fhd-quiet-blue-color.jpg?v=638539797551400000', 'https://exitocol.vtexassets.com/arquivos/ids/23178712/portatil-asus-intel-core-i5-1235u-ram-16gb-ssd-1tb-pantalla-156-fhd-quiet-blue-color.jpg?v=638539797552500000', 'https://exitocol.vtexassets.com/arquivos/ids/23178713/portatil-asus-intel-core-i5-1235u-ram-16gb-ssd-1tb-pantalla-156-fhd-quiet-blue-color.jpg?v=638539797554070000', 'https://exitocol.vtexassets.com/arquivos/ids/23178714/portatil-asus-intel-core-i5-1235u-ram-16gb-ssd-1tb-pantalla-156-fhd-quiet-blue-color.jpg?v=638539797556270000', 'https://exitocol.vtexassets.com/arquivos/ids/23178710/portatil-asus-intel-core-i5-1235u-ram-16gb-ssd-1tb-pantalla-156-fhd-quiet-blue-color.jpg?v=638539797549700000

Go on the Computer Page

In [110]:
category = "Tecnología"
article_type_choice = "Portátiles"

main_page=driver.current_url
def go_to_article_page(driver, category, article_type):
    import time
    from selenium.webdriver.common.by import By

    try:
        # Récupère le menu des catégories
        categories_menu = driver.find_element(By.XPATH, '//*[@id="header-page"]/aside/ul/section[2]')

        # Cherche tous les div ayant l'attribut data-link-content="true"
        divs_with_data_link_content = categories_menu.find_elements(By.XPATH, './/div[@data-link-content="true"]')

        # Clique sur la catégorie principale
        for div in divs_with_data_link_content:
            if div.text.strip() == category:
                div.click()
                print(f"Catégorie '{category}' cliquée.")
                time.sleep(1)
                break

        # Menu déroulé : chercher le type d'article
        selected_menu = driver.find_element(By.XPATH, './/ul[@data-content-list="true"]')
        list_elements = selected_menu.find_elements(By.TAG_NAME, 'li')

        for li in list_elements:
            if li.text.strip() == article_type:
                try:
                    link = li.find_element(By.TAG_NAME, 'a')
                    link.click()
                    print(f"Lien cliqué vers la page '{article_type}'")
                    time.sleep(2)
                    return driver.current_url
                except:
                    print(f"Lien introuvable dans l'élément '{article_type}'")
                    return None
        print(f"Type d'article '{article_type}' non trouvé.")
        return None

    except Exception as e:
        print(f"Erreur lors de la navigation : {e}")
        return None
try:
    go_to_article_page(driver, category, article_type_choice)
except Exception as e:
    print(str(e))

Catégorie 'Tecnología' cliquée.
Lien cliqué vers la page 'Portátiles'


Getting all the Checkboxes

In [ ]:
computer_filter_menu = driver.find_element(By.XPATH, "//div[@class='accordion_fs-accordion__mi4MR']")

# Récupère tous les titres des filtres (balises h4)
filters = computer_filter_menu.find_elements(By.TAG_NAME, 'h4')
Sub_categoría = filters[1]

# desktop-store-filter-Sub-categoría-Accesorios de cámaras

# Vérifier si l'élément Sub_categoría a été trouvé
if Sub_categoría:
    # Trouver toutes les checkboxes à l'intérieur de 'Sub_categoría'
    checkboxes = Sub_categoría.find_elements(By.XPATH, '//input[@type="checkbox" and @data-fs-checkbox="true"]')
    print(f"Nombre de checkboxes trouvées : {len(checkboxes)}")
    for checkbox in checkboxes:
        print(checkbox.get_attribute("data-value"))
else:
    print("L'élément 'Sub-categoría' n'a pas été trouvé.")

In [125]:
def cocher_checkboxes(filters_choice, checkboxes, driver):
    # Normalisation des filtres
    filters_normalized = [unidecode(item.lower()) for item in filters_choice]

    for i in range(len(checkboxes)):
        try:
            # On relocalise à chaque boucle pour éviter stale
            checkboxes_refreshed = driver.find_elements(By.CSS_SELECTOR, "input[type='checkbox'][data-value]")
            if i >= len(checkboxes_refreshed):
                break  # Au cas où le DOM change

            checkbox = checkboxes_refreshed[i]

            data_value_raw = checkbox.get_attribute("data-value")
            if data_value_raw is None:
                continue

            data_value = unidecode(data_value_raw.replace("-", " ").lower())
            matched = any(f in data_value for f in filters_normalized)

            if matched and not checkbox.is_selected():
                click_with_JS(checkbox, driver)
                print(f"[CHECK] Checkbox pour '{data_value}' cochée.")

            elif not matched and checkbox.is_selected():
                click_with_JS(checkbox, driver)
                print(f"[UNCHECK] Checkbox pour '{data_value}' décochée.")
        
        except Exception as e:
            print(f"[ERROR] Problème sur checkbox {i}: {e}")
            continue

from unidecode import unidecode


In [123]:
queries = [
    "asus", "8 gb", "16 GB", "nvidia", "amd", "24 gb", "32 gb", "ssd",
    "ryzen 7", "500 GB", "1 TB", "i7", "gaming", 'portatiles'
]

In [126]:
try:
    cocher_checkboxes(filters_choice=queries, checkboxes=checkboxes,driver=driver)
except Exception as e:
    print(str(e))

[CHECK] Checkbox pour 'computadores gaming' cochée.
[CHECK] Checkbox pour 'portatiles' cochée.
[CHECK] Checkbox pour 'amd' cochée.
[CHECK] Checkbox pour 'astro gaming' cochée.
[CHECK] Checkbox pour 'asus' cochée.


In [38]:
from currencies import Currency

class CurrencyConverter:
    def __init__(self, from_currency, to_currency, exchange_rate):
        self.from_currency = Currency(from_currency)
        self.to_currency = Currency(to_currency)
        self.exchange_rate = exchange_rate

    def convert(self, amount):
        # Effectuer la conversion de l'euro vers le COP
        amount_in_target_currency = amount * self.exchange_rate
        # Retourne un float sans formatage en chaîne
        return amount_in_target_currency

# Exemple d'utilisation :
# Taux de conversion (peut être récupéré depuis une API, ici 1 EUR = 4500 COP)
exchange_rate_eur_to_cop = 4500

# Création d'un convertisseur de devise
converter_eur_cop = CurrencyConverter('EUR', 'COP', exchange_rate_eur_to_cop)


In [82]:
# Exemple d'utilisation
min_value = converter_eur_cop.convert(300)  # Conversion de 300 EUR
max_value = converter_eur_cop.convert(500)  # Conversion de 450 EUR

In [105]:
def set_slider_value(driver, min_value, max_value):
    # Récupérer la valeur actuelle des deux sliders avant de les changer
    initial_left_value = driver.execute_script("""
    const leftSlider = document.querySelector('input[type="range"][data-slider-thumb="left"]');
    return leftSlider ? leftSlider.value : null;
    """)
    
    initial_right_value = driver.execute_script("""
    const rightSlider = document.querySelector('input[type="range"][data-slider-thumb="right"]');
    return rightSlider ? rightSlider.value : null;
    """)

    print(f"Valeur initiale du slider gauche : {initial_left_value}")
    print(f"Valeur initiale du slider droit : {initial_right_value}")

    # Changer la valeur des sliders via JavaScript
    script = """
    const leftSlider = document.querySelector('input[type="range"][data-slider-thumb="left"]');
    const rightSlider = document.querySelector('input[type="range"][data-slider-thumb="right"]');
    
    if (leftSlider) {
        leftSlider.value = arguments[0]; // Définir la nouvelle valeur du slider gauche
        leftSlider.dispatchEvent(new Event('input', { bubbles: true })); // Simuler l'événement 'input'
        leftSlider.dispatchEvent(new Event('change', { bubbles: true })); // Simuler l'événement 'change'
    }

    if (rightSlider) {
        rightSlider.value = arguments[1]; // Définir la nouvelle valeur du slider droit
        rightSlider.dispatchEvent(new Event('input', { bubbles: true })); // Simuler l'événement 'input'
        rightSlider.dispatchEvent(new Event('change', { bubbles: true })); // Simuler l'événement 'change'
    }
    """
    # Exécuter le script pour changer les valeurs des deux sliders
    driver.execute_script(script, min_value, max_value)

    # Récupérer la nouvelle valeur des deux sliders après modification
    final_left_value = driver.execute_script("""
    const leftSlider = document.querySelector('input[type="range"][data-slider-thumb="left"]');
    return leftSlider ? leftSlider.value : null;
    """)

    final_right_value = driver.execute_script("""
    const rightSlider = document.querySelector('input[type="range"][data-slider-thumb="right"]');
    return rightSlider ? rightSlider.value : null;
    """)

    print(f"Valeur après modification du slider gauche : {final_left_value}")
    print(f"Valeur après modification du slider droit : {final_right_value}")

    # Comparer les valeurs avant et après
    if initial_left_value != final_left_value:
        print(f"Le slider gauche a été mis à jour avec succès. Nouvelle valeur : {final_left_value}")
    else:
        print(f"Aucune modification n'a été effectuée sur le slider gauche. Valeur actuelle : {final_left_value}")

    if initial_right_value != final_right_value:
        print(f"Le slider droit a été mis à jour avec succès. Nouvelle valeur : {final_right_value}")
    else:
        print(f"Aucune modification n'a été effectuée sur le slider droit. Valeur actuelle : {final_right_value}")


In [106]:
set_slider_value(driver, min_value, max_value)

Valeur initiale du slider gauche : 38000
Valeur initiale du slider droit : 38000
Valeur après modification du slider gauche : 1350000
Valeur après modification du slider droit : 2250000
Le slider gauche a été mis à jour avec succès. Nouvelle valeur : 1350000
Le slider droit a été mis à jour avec succès. Nouvelle valeur : 2250000


In [107]:
aplicar_filtros = driver.find_element(By.XPATH, "//button[normalize-space()='Aplicar filtros']")
aplicar_filtros.click()